# Making the initial timetable

In [151]:
import pandas as pd
import numpy as np

In [152]:
proc_tt = pd.read_excel("Processed Data/encoded_timetable_data.xlsx").copy()
print(proc_tt.columns)
print("Shape of dataframe: ", proc_tt.shape)

Index(['Course', 'Code', 'Linked Course', 'Activity', 'Activity Type Name',
       'Planned Size', 'Real Size', 'Delivery Semester',
       'Teaching Week Pattern', 'Number Of Teaching Weeks', 'Scheduled Days',
       'Scheduled Start Time', 'Scheduled End Time', 'Duration', 'Total Hours',
       'Zone Name', 'room_name', 'Delivery Period', 'Normal Year Taken',
       'Degrees Compulsory For', 'Categories', 'enrollment_number', 'building',
       'capacity', 'encoded_days', 'Abbreviated Course Name', 'time_periods',
       'duration_hours', 'total_duration_hours', 'course_classtype',
       'ws_groups', 'no_of_agg_ws_groups', 'aggregated_ws_groups',
       'sem1_pattern_one_hot', 'sem2_pattern_one_hot'],
      dtype='object')
Shape of dataframe:  (1259, 35)


## Cleaning data

In [153]:
# Drop MIGS course as we are not considering it for SoM
proc_tt.drop(proc_tt.index[proc_tt.Code == "MATH11220"].tolist(), inplace = True)

## Choosing weeks in semester 1 and semester 2 for timetabling

In [154]:
sem1_tt = proc_tt[proc_tt["sem1_pattern_one_hot"].str.contains('1')]
sem2_tt = proc_tt[proc_tt["sem2_pattern_one_hot"].str.contains('1')]
s2w3_tt = sem2_tt[sem2_tt["Teaching Week Pattern"].str.contains('s2w3')]
s2w4_tt = sem2_tt[sem2_tt["Teaching Week Pattern"].str.contains('s2w4')]

In [155]:
days = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday"]
periods = ['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18']

### Create initial timetable for any week

In [156]:
def create_init_tt(sem, week, days, periods, full_tt = proc_tt):
    sem_tt = full_tt[full_tt["sem"+str(sem)+"_pattern_one_hot"].str.contains('1')]
    sw_tt = sem_tt[sem_tt["Teaching Week Pattern"].str.contains('s'+str(sem)+'w'+str(week))]
    init_tt = pd.DataFrame(np.zeros((18, 5), dtype = "O"), index = periods, columns = days)
    
    # # For cross checking
    sw_copy = sw_tt.copy()
    sw_copy["time_periods"] = sw_copy["time_periods"].str.split(', ')
    sw_copy = sw_copy.explode("time_periods")
    # sw_copy.reset_index(drop=True, inplace=True)

    count = 0

    for d, day in enumerate(days):
        sw_check = sw_copy[sw_copy["Scheduled Days"] == day]
        day_tt = sw_check.drop_duplicates(subset = ["course_classtype", "room_name", "time_periods"])
        count_check = 0
        indices = []
        for t, period in enumerate(periods):
            print(day, period)
            sw_day_time_tt = sw_copy[(sw_copy["Scheduled Days"] == day) & (sw_copy["time_periods"] == period)]
            sw_dt_tt_unique = sw_day_time_tt.drop_duplicates(subset = ["course_classtype", "room_name"])
            count_check += sw_dt_tt_unique.shape[0]
            unique_indices = sw_dt_tt_unique.index.tolist()
            indices.extend(unique_indices)
            classes_rooms = []
            for row in sw_dt_tt_unique.itertuples():
                classes_rooms.append([row.course_classtype, row.room_name])
                count += 1
            
            init_tt.loc[period, day] = str(classes_rooms).replace(", ", "")

        print(count_check, day_tt.shape[0])
        # print(np.sort(np.array(indices)), np.sort(np.array(day_tt.index.tolist())))

    print(count, sw_copy.drop_duplicates(subset = ["course_classtype", "room_name", "Scheduled Days", "time_periods"]).shape[0])
    return init_tt
    
init_tt_s2w4 = create_init_tt(sem=2, week=4, days=days, periods=periods)
init_tt_s2w3 = create_init_tt(sem=2, week=3, days=days, periods=periods)

init_tt_s2w4.T.to_csv(f"Initial_timetables/init_tt_s2w4.csv")
init_tt_s2w3.T.to_csv(f"Initial_timetables/init_tt_s2w3.csv")

Monday 1
Monday 2
Monday 3
Monday 4
Monday 5
Monday 6
Monday 7
Monday 8
Monday 9
Monday 10
Monday 11
Monday 12
Monday 13
Monday 14
Monday 15
Monday 16
Monday 17
Monday 18
100 100
Tuesday 1
Tuesday 2
Tuesday 3
Tuesday 4
Tuesday 5
Tuesday 6
Tuesday 7
Tuesday 8
Tuesday 9
Tuesday 10
Tuesday 11
Tuesday 12
Tuesday 13
Tuesday 14
Tuesday 15
Tuesday 16
Tuesday 17
Tuesday 18
104 104
Wednesday 1
Wednesday 2
Wednesday 3
Wednesday 4
Wednesday 5
Wednesday 6
Wednesday 7
Wednesday 8
Wednesday 9
Wednesday 10
Wednesday 11
Wednesday 12
Wednesday 13
Wednesday 14
Wednesday 15
Wednesday 16
Wednesday 17
Wednesday 18
74 74
Thursday 1
Thursday 2
Thursday 3
Thursday 4
Thursday 5
Thursday 6
Thursday 7
Thursday 8
Thursday 9
Thursday 10
Thursday 11
Thursday 12
Thursday 13
Thursday 14
Thursday 15
Thursday 16
Thursday 17
Thursday 18
122 122
Friday 1
Friday 2
Friday 3
Friday 4
Friday 5
Friday 6
Friday 7
Friday 8
Friday 9
Friday 10
Friday 11
Friday 12
Friday 13
Friday 14
Friday 15
Friday 16
Friday 17
Friday 18
68 68
4